# Provision Scripts

## References
- Using CLI: https://chameleoncloud.readthedocs.io/en/latest/technical/cli.html#the-openstack-rc-script
- Using Container API: https://python-chi.readthedocs.io/en/latest/modules/container.html
- Webinar: https://www.youtube.com/watch?v=1MPROv595LM

## Authentication
Before starting `jupyter-lab`, you need to `source openrc.sh` and enter in your password. This file must be downloaded from the Chameleon GUI. 

## Code

In [ ]:
import logging
import pprint
from pathlib import Path
from typing import List

import chi
from chi import container
from chi import lease

logging.basicConfig(level=logging.INFO)
pp = pprint.PrettyPrinter(indent=2)

In [ ]:
# initial setup
chi.use_site("CHI@Edge")
chi.set("project_name", "CHI-210827")
chi.set("project_domain_name", "chameleon")

In [ ]:
# util to reconfigure condor daemons once central manager ip is known
def configure_condor(container_id: str, configs: List[str]) -> None:
    print("the following configs will be sent to container: {} /etc/condor/config.d/60-condor.conf".format(container_id))
    print(*configs, sep="\n")
    
    # create a new config file and upload
    conf_dir = Path("tmp")
    conf_dir.mkdir()
    conf_file = conf_dir / "60-condor.conf"
    
    with conf_file.open("w") as f:
        for config in configs:
            f.write(config + "\n")
    
    resp = container.upload(container_id, str(conf_dir), "/etc/condor/config.d")
    print(resp)
    assert resp[0].status_code == 200
    
    print("config upload successful..")
    
    # invoke condor_reconfig
    resp = container.execute(container_id, "condor_reconfig")
    print(resp["output"])
    assert resp["exit_code"] == 0
    
    print("condor_reconfig cmd successful..")
    

### Setup Central Manager

In [ ]:
# create a lease for a central manager
start, end = lease.lease_duration(days=1)
reservations = []
lease.add_device_reservation(reservations, count=1, device_model="4")
container_lease = lease.create_lease("condor-central-manager", reservations)
cm_lease_id = container_lease["id"]

print("Waiting for lease to start ...")
lease.wait_for_active(cm_lease_id)
print("condor-central-manager lease_id: {}".format(cm_lease_id))
print("Done!")

In [ ]:
# start condor central manager container
cm_cont = container.create_container(
                name="central-manager",
                image="ryantanaka/condor8-arm64-central-manager",
                exposed_ports=["9618"],
                reservation_id="84bd7c9b-723c-44aa-ba10-9e02dd6b10d9"
            )

print("waiting for central-manager container to start")
container.wait_for_active(cm_cont.uuid, timeout=600)
print("central-manager container_id: {}".format(cm_cont.uuid))
print("done!")

In [ ]:
central_manager_public_ip = container.associate_floating_ip(cm_cont.uuid)
print(central_manager_public_ip)

In [ ]:
# reconfigure central manager with host ip
CONDOR_HOST_CONFIG = "CONDOR_HOST = {}".format(central_manager_public_ip)
configure_condor(cm_cont.uuid, [CONDOR_HOST_CONFIG])

In [ ]:
# check to see if configs were updated
resp = container.execute(cm_cont.uuid, "condor_config_val -dump CONDOR_HOST")
print(resp["output"])

In [ ]:
# this will fail due to timeout, simpler commands like `condor_status -help` will work however
resp = container.execute(cm_cont.uuid, "condor_status")
print(resp["output"])

### Setup Worker

In [ ]:
# create a lease for a worker
start, end = lease.lease_duration(days=1)
reservations = []
lease.add_device_reservation(reservations, count=1, device_model="4")
container_lease = lease.create_lease("condor-worker", reservations)
worker_lease_id = container_lease["id"]

print("Waiting for lease to start ...")
lease.wait_for_active(worker_lease_id)
print("condor-worker lease_id: {}".format(worker_lease_id))
print("Done!")

In [ ]:
# start condor worker
worker_cont = container.create_container(
                    name="worker",
                    image="ryantanaka/condor8-arm64-worker",
                    reservation_id=worker_lease_id
                )


print("waiting for worker container to start")
container.wait_for_active(worker_cont.uuid, timeout=600)
print("worker container_id: {}".format(worker_cont.uuid))
print("done!")

# TODO: need to handle error when container can't properly start
# TODO: need to handle error when worker can't connect to cm (add in timeout..)

In [ ]:
# reconfigure worker with central manager ip
CONDOR_HOST_CONFIG = "CONDOR_HOST = {}".format(central_manager_public_ip)
configure_condor(worker_cont.uuid, [CONDOR_HOST_CONFIG])